In [ ]:
"""
Acknowledgement
---------------
1. set your browser download location to your desire location because all files will be downloaded to the download location setted in your browser.
2. create these files -> files_info.json, bot.log, .secret

Thanks -> pythonjar, MariyaSha and some other stackoverflow members...

"""
# Packages & Libraries
import json
from json.decoder import JSONDecodeError
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support import expected_conditions as EC

# Custom imports
from constantsAndVariables import (
    secrets,
    encoding,
    target_file_type,
    tracker_file_location,
)
from helperFunctions import (
    login,
    updateLog,
    isDownloaded,
    waitNSeconds,
    downloadFile,
    loadMoreFiles,
    appendFilesInfo,
    initializeWebpage,
    initializeWebDriver,
    waitToFinishDownload,
    getExistingFilesInfo,
)


In [ ]:
driver, wait = initializeWebDriver()
login(driver, wait)


In [ ]:
initializeWebpage(driver, "https://www.facebook.com/groups/201623576939858/files/")

"""
 targets identifiers -> this identifiers will change continuously so update it according to your needs 

"""
download_button_cssSelector = "a[href*='https://www.facebook.com/download/']"
fileOption_xpath = "//div[@aria-label='File options']"  # by default finds 15 per-scroll

# Target identifiers
# They change too often so they have to be present here
fileName_xpath = "//span[@class='d2edcug0 hpfvmrgz qv66sw1b c1et5uql b0tq1wua a8c37x1j fe6kdd0r mau55g9w c8b282yb keod5gw0 nxhoafnm aigsh9s9 d9wwppkn hrzyx87i jq4qci2q a3bd9o3v lrazzd5p oo9gr5id hzawbc8m']"  # find 15 per-scroll
fileTypeDate_xpath = "//span[@class='d2edcug0 hpfvmrgz qv66sw1b c1et5uql b0tq1wua a8c37x1j fe6kdd0r mau55g9w c8b282yb keod5gw0 nxhoafnm aigsh9s9 tia6h79c iv3no6db e9vueds3 j5wam9gi b1v8xokw oo9gr5id hzawbc8m']"  # find 30 per-scroll

In [ ]:
j, k = 0, 1

registered_files_count = 0

# Refresh existing files info every time this cell loads
downloaded_files, tracked_files = getExistingFilesInfo()

# Total files registered in the tracker file(files_info.json)
if tracked_files:
    with open(tracker_file_location, "r", encoding=encoding) as f:
        try:
            # Last registered file id
            registered_files_count = int(json.load(f)["files"][-1]["id"])
        except JSONDecodeError:
            pass

try:
    # Grabbing initial loadeded target
    file_option_buttons = wait.until(
        EC.presence_of_all_elements_located((By.XPATH, fileOption_xpath))
    )
    file_names = wait.until(
        EC.presence_of_all_elements_located((By.XPATH, fileName_xpath))
    )
    file_types_and_dates = wait.until(
        EC.presence_of_all_elements_located((By.XPATH, fileTypeDate_xpath))
    )
except TimeoutException:
    print("XPATH's identifier has changed again! 🫡")


for idx, button in enumerate(file_option_buttons):
    try:
        _name = file_names[idx].text
        _type = file_types_and_dates[j].text
        _date = file_types_and_dates[k].text

        # as 'file_types_and_dates' contains both file-type and date
        j += 2
        k += 2

        # Scrolling after it reaches at the end of the list to load more files
        if button is file_option_buttons[-1]:
            [file_option_buttons, file_names, file_types_and_dates] = loadMoreFiles(
                driver,
                [file_option_buttons, file_names, file_types_and_dates],
                [fileOption_xpath, fileName_xpath, fileTypeDate_xpath],
            )

            updateLog("\nTotal Loaded Files: {}\n".format(len(file_option_buttons)))

        """
        Download pdf files only
        Cannot check this before the scroll because
        here is a possibility to have more than one page worth of non-pdf files
        
        """
        if not _type == target_file_type.upper():
            updateLog(
                '\n😪 Skipping: "{}", 🤔 Reason: FILE_TYPE: "{}"'.format(_name, _type)
            )
            continue

        should_download, update_tracker_file = isDownloaded(
            _name, _date, downloaded_files, tracked_files
        )
        if should_download:
            updateLog('\n😪 Skipping: "{}", 🤔 Reason: ALREADY_DOWNLOADED'.format(_name))
            continue

        """
        If everything is OK then initializing file download 😁
        
        wait.until(EC.element_to_be_clickable(button)).click()
        -> this was giving me -> ElementClickInterceptedException

        """
        # Clicking the more ('...' 3 dot) button
        driver.execute_script("arguments[0].click();", button)

        waitNSeconds()

        # Try to download the file
        registered_files_count = downloadFile(
            driver, wait, download_button_cssSelector, registered_files_count
        )

        """
        Updating tracker file after a successful file download 😎
        This should be called before the 'waitToFinishDownload' or else there can be a situation 
        where the file has been downloaded successfully but it was not registered

        """
        if update_tracker_file:
            appendFilesInfo(
                {
                    "id": registered_files_count,
                    "type": _type,
                    "name": _name,
                    "uploaded_date": _date,
                }
            )

        """
        Waiting until the file gets downloaded completely
        We have to wait for the file to be downloaded, if we continue without it there maybe some files
        that couldn't be downloaded on time (before the link expire) and later on those files can't be resumed
        
        """
        download_time = waitToFinishDownload(secrets[2], registered_files_count)

        # Waiting 600s before stoping the Bot for slow/no internet 😟
        if download_time == -1:
            break

        """ Waiting at least 2s before every download request for safety 😅 """

        # Updating info after a successful file download 😎
        updateLog('✔ DONE: "{}", "{}", "{}"'.format(_type, _date, _name))

        break

    except Exception as e:
        updateLog("\n*** ERROR at {}, date: {} ***\n--> {}\n".format(_name, _date, e))


# driver.close() # not closing because I had to run this cell multiple times during test
updateLog(
    "\n\n\n👏👏👏 Completed 👏👏👏\n\nFiles downloaded: {} 😉\n".format(registered_files_count)
)
updateLog(
    "Go ahead and give this https://github.com/ShahriarDhruvo/Bots a star, thanks\t--- SED"
)
